Generación de textos con 🤗 Transformers

In [ ]:
!pip install transformers datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 31.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
from IPython.display import HTML

def display_df(df, max_cols=15, header=True, index=True):
    return display(HTML(df.to_html(header=header,index=index, max_cols=max_cols)))

# Greedy search decoding

El método de decodificación más sencillo para obtener tokens discretas a partir de la salida continua de un modelo es seleccionar con "avidez" la token con mayor probabilidad en cada paso de tiempo.


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

device = "cuda" if torch.cuda.is_available() else "cpu"
model_name = "mrm8488/spanish-gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
# Añadimos el token EOS como token PAD para evitar warnings
model = AutoModelForCausalLM.from_pretrained(model_name, pad_token_id=tokenizer.eos_token_id).to(device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/226 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/846k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/505k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.45M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/883 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/510M [00:00<?, ?B/s]

In [ ]:
def model_size(model):
    return sum(t.numel() for t in model.parameters())

print(f"Tamaño del GPT español: {model_size(model)/1000**2:.1f}M parámetros")

Tamaño del GPT español: 124.4M parámetros


In [ ]:
import pandas as pd

# Aquí se indica el "prompt" para continuar
input_txt = "El amor es eterno mientras dura. "

input_ids = tokenizer(input_txt, return_tensors="pt")["input_ids"].to(device)
iterations = []
n_steps = 12
choices_per_step = 5

with torch.no_grad():
    for _ in range(n_steps):
        iteration = dict()
        iteration["Input"] = tokenizer.decode(input_ids[0])
        output = model(input_ids=input_ids)
        # Seleccionar los logits del primer batch y del último token y aplicar softmax
        next_token_logits = output.logits[0, -1, :]
        next_token_probs = torch.softmax(next_token_logits, dim=-1)
        sorted_ids = torch.argsort(next_token_probs, dim=-1, descending=True)
        # Almacenar las tokens con mayores probabilidades
        for choice_idx in range(choices_per_step):
            token_id = sorted_ids[choice_idx]
            token_prob = next_token_probs[token_id].cpu().numpy()
            token_choice = (
                f"{tokenizer.decode(token_id)} ({100 * token_prob:.2f}%)"
            )
            iteration[f"Choice {choice_idx+1}"] = token_choice
        # Añadir el siguiente token previsto a los inputs
        input_ids = torch.cat([input_ids, sorted_ids[None, 0, None]], dim=-1)
        iterations.append(iteration)

display_df(pd.DataFrame.from_records(iterations), index=None)

Input,Choice 1,Choice 2,Choice 3,Choice 4,Choice 5
El amor es eterno mientras dura.,Ésta (38.28%),________________ (16.74%),{\ (6.56%),Quienes (6.47%),________________________________ (4.61%)
El amor es eterno mientras dura. Ésta,es (84.32%),no (2.89%),será (1.04%),era (0.84%),fue (0.82%)
El amor es eterno mientras dura. Ésta es,la (54.23%),una (13.19%),mi (11.13%),tu (5.73%),nuestra (3.97%)
El amor es eterno mientras dura. Ésta es la,historia (12.52%),verdad (3.90%),última (2.13%),vida (2.09%),única (1.84%)
El amor es eterno mientras dura. Ésta es la historia,de (75.85%),del (11.07%),que (2.16%),. (1.31%),más (1.06%)
El amor es eterno mientras dura. Ésta es la historia de,un (27.52%),una (17.28%),la (5.55%),amor (4.31%),dos (4.27%)
El amor es eterno mientras dura. Ésta es la historia de un,amor (29.85%),hombre (17.66%),joven (7.53%),chico (4.98%),muchacho (2.07%)
El amor es eterno mientras dura. Ésta es la historia de un amor,que (34.47%),eterno (5.27%),. (5.02%),verdadero (4.23%),duradero (3.16%)
El amor es eterno mientras dura. Ésta es la historia de un amor que,dura (25.91%),duró (10.71%),se (8.82%),perdura (6.36%),no (5.98%)
El amor es eterno mientras dura. Ésta es la historia de un amor que dura,para (56.40%),toda (8.97%),. (7.61%),eternamente (5.84%),y (2.51%)


In [ ]:
input_ids = tokenizer(input_txt, return_tensors="pt")["input_ids"].to(device)
output = model.generate(input_ids, max_length=20)
print(tokenizer.decode(output[0]))

El amor es eterno mientras dura. Ésta es la historia de un amor que dura para siempre.


Vamos a intentar reproducir la historia inventada del Unicornio

In [ ]:
max_length = 128
input_txt ="""En un hallazgo sorprendente, los científicos descubrieron una manada de unicornios \
que vivía en un valle remoto, hasta ahora inexplorado, en la cordillera de los Andes. \
Más sorprendente aún para los investigadores fue el hecho de que los unicornios hablaban \
un inglés perfecto. """
input_ids = tokenizer(input_txt, return_tensors="pt")["input_ids"].to(device)
output_greedy = model.generate(input_ids, max_length=max_length,
                               do_sample=False)
print(tokenizer.decode(output_greedy[0]))

En un hallazgo sorprendente, los científicos descubrieron una manada de unicornios que vivía en un valle remoto, hasta ahora inexplorado, en la cordillera de los Andes. Más sorprendente aún para los investigadores fue el hecho de que los unicornios hablaban un inglés perfecto. ˇEl idioma de los unicornios!ˇEl idioma de los unicornios!ˇEl idioma de los unicornios!ˇEl idioma de los unicornios!ˇEl idioma de los unicornios!ˇEl idioma de los unicornios!ˇEl idioma de los unicornios!ˇ


# Beam search decoding
beam-search.png

En lugar de decodificar el token con la mayor probabilidad en cada paso, beam search mantiene un registro de los próximos tokens más probables, donde
 se refiere al número de beams o hipótesis parciales.

Ambas opciones se ven afectadas por el texto repetitivo. Una forma de solucionar esto es imponer una penalización de n-gramas:

In [ ]:
output_beam = model.generate(input_ids, max_length=max_length, num_beams=5,
                             do_sample=False, no_repeat_ngram_size=2)
#logp = sequence_logprob(model, output_beam, input_len=len(input_ids[0]))
print(tokenizer.decode(output_beam[0]))
#print(f"\nlog-prob: {logp:.2f}")

En un hallazgo sorprendente, los científicos descubrieron una manada de unicornios que vivía en un valle remoto, hasta ahora inexplorado, en la cordillera de los Andes. Más sorprendente aún para los investigadores fue el hecho de que los unicornios hablaban un inglés perfecto. Ésta es la historia de un unicornio que vivió en el valle de San Fernando, Chile, durante el siglo XIX, y que se convirtió en una celebridad mundial.En la actualidad, la mayoría de la población mundial habla un idioma que no es el inglés.El idioma más hablado en todo el mundo, el español, es una de las lenguas más habladas


# Sampling methods
# Temperatura
Podemos controlar fácilmente la diversidad de la salida añadiendo un parámetro de temperatura que reescala los logits antes de tomar el softmax. Determina cuánto de creativo debe ser el modelo, controla la aleatoriedad de la salida. 0 determinista, 5 más creativo aplana las probababilidades de las palabras a seleccionar.

In [ ]:
torch.manual_seed(42)
output_temp = model.generate(input_ids, max_length=max_length, do_sample=True,
                             temperature=0.75, top_k=0)
print(tokenizer.decode(output_temp[0]))

En un hallazgo sorprendente, los científicos descubrieron una manada de unicornios que vivía en un valle remoto, hasta ahora inexplorado, en la cordillera de los Andes. Más sorprendente aún para los investigadores fue el hecho de que los unicornios hablaban un inglés perfecto. ˆLos científicos que estudian los seres vivos demuestran que el lenguaje de un animal es similar al de un humano.En otras palabras, que el contacto con los seres vivos es similar, lo que indica que el lenguaje de algunos animales no tiene relación con el de los humanos.No obstante, en gran parte del mundo el inglés es una lengua muerta, incluso cuando


Podemos ver claramente que una temperatura alta de 2.0 ha producido un galimatías; al acentuar las tokens raras, hemos hecho que el modelo cree una gramática extraña y bastantes palabras inventadas. Veamos qué ocurre si reducimos la temperatura a 0.75

# Top-k y top-p sampling
El muestreo top-k y el núcleo (top-p) son dos alternativas o extensiones populares al uso de la temperatura. En ambos casos, la idea básica es restringir el número de tokens posibles de los que podemos tomar muestras en cada paso de tiempo:
 - Top p. Suma probabilidades y se queda con las primeras opciones que superen el "p" %
 - Top k. Se queda con los "k" primeros tokens con mayor probabilidad y de esos elige uno.

In [ ]:
torch.manual_seed(42)
output_topk = model.generate(input_ids, max_length=max_length, do_sample=True,
                             top_k=200)
print(tokenizer.decode(output_topk[0]))

En un hallazgo sorprendente, los científicos descubrieron una manada de unicornios que vivía en un valle remoto, hasta ahora inexplorado, en la cordillera de los Andes. Más sorprendente aún para los investigadores fue el hecho de que los unicornios hablaban un inglés perfecto. ue la mayoría de los depredadores aéreos no entienden la palabra “viven”.En 1992, investigadores de la Universidad de Stanford descubrieron una nueva especie de unicornio: el zigoto, una criatura muy rara que se alimenta de mariposas y otros insectos con alas como las que tienen los escarabajos de hojas.Desde el descubrimiento de fósiles notables de vida en el


In [ ]:
torch.manual_seed(42)
output_topp = model.generate(input_ids, max_length=max_length, do_sample=True,
                             top_p=0.50)
print(tokenizer.decode(output_topp[0]))

En un hallazgo sorprendente, los científicos descubrieron una manada de unicornios que vivía en un valle remoto, hasta ahora inexplorado, en la cordillera de los Andes. Más sorprendente aún para los investigadores fue el hecho de que los unicornios hablaban un inglés perfecto. ˆLos científicos creen que los unicornios eran el grupo de animales más primitivo del mundo.La palabra unicornio significa literalmente "el unicornio".Y es una referencia a los unicornios.El unicornio es un animal muy parecido a los unicornios.Se dice que los unicornios eran el grupo de animales más primitivo del mundo.
